In [59]:
# Import library

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

In [60]:
# Load images

def load_image(PATH):
    images = []
    for fileName in os.listdir(PATH):
        image = cv2.imread(PATH + fileName)
        if image is not None:
            images.append(image)
    return images

next_images = load_image("Dataset/Next/")
back_images = load_image("Dataset/Back/")

print("Loaded:", len(next_images), "next,", len(back_images), "back")

Loaded: 10 next, 10 back


In [61]:
# HOG descriptor

hog = cv2.HOGDescriptor(
    _winSize=(128, 128),
    _blockSize=(32, 32),
    _blockStride=(16, 16),
    _cellSize=(16, 16),
    _nbins=9
)

def compute_canny_hog(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_resized = cv2.resize(image_gray, (128, 128))
    edges = cv2.Canny(image_resized, 50, 150)
    hog_features = hog.compute(edges).flatten()

    return hog_features

In [62]:
# Extract features and labels

X = []
y = []

for image in next_images:
    X.append(compute_canny_hog(image))
    y.append(1)

for image in back_images:
    X.append(compute_canny_hog(image))
    y.append(0)

X = np.array(X)
y = np.array(y)

X.shape

(20, 1764)

In [63]:
# Train SVM model

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM classifier
svm = LinearSVC()
svm.fit(X_scaled, y)

LinearSVC()

In [64]:
# Prediction function

def predict(image):
    feat = compute_canny_hog(image)
    feat = feat.reshape(1, -1)
    
    feat = scaler.transform(feat)
    label = svm.predict(feat)[0]
    
    return "NEXT" if label == 1 else "BACK"


In [65]:
# Testing

test_image = cv2.imread("test.jpg")
label = predict(test_image)
print(label)

NEXT
